In [52]:
import pandas as pd
from sodapy import Socrata
import os

In [53]:
# Extract environment variables
APP_TOKEN = os.getenv('APP_TOKEN')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')

In [54]:
# Define which client/dataset to use 
CLIENT_IDENTIFIER="data.cityofnewyork.us"
DATASET_IDENTIFIER="vfnx-vebw"

# Limit for # of rows 
# limit=2000

In [55]:
# Get data with app token to avoid data limits on call 
client = Socrata(CLIENT_IDENTIFIER, APP_TOKEN, username=USERNAME, password=PASSWORD)
results = client.get(DATASET_IDENTIFIER)#, limit=limit)

In [56]:
# Convert data to Pandas dataframe
results_df = pd.DataFrame.from_records(results)
column_names=results_df.columns.values.tolist()

In [57]:
print(results_df.columns.values.tolist())

['x', 'y', 'unique_squirrel_id', 'hectare', 'shift', 'date', 'hectare_squirrel_number', 'combination_of_primary_and', 'running', 'chasing', 'climbing', 'eating', 'foraging', 'kuks', 'quaas', 'moans', 'tail_flags', 'tail_twitches', 'approaches', 'indifferent', 'runs_from', 'geocoded_column', ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih', 'primary_fur_color', 'location', 'above_ground_sighter', 'age', 'color_notes', 'highlight_fur_color', 'specific_location', 'other_activities', 'other_interactions', ':@computed_region_efsh_h5xi']


In [58]:
# Squirrels per hectare
squirrels_per_hectare=results_df.groupby('hectare')['unique_squirrel_id'].count().reset_index(name="squirrel_count")
print(squirrels_per_hectare)


    hectare  squirrel_count
0       01A               1
1       01B               5
2       01C               3
3       01D               5
4       01E               1
..      ...             ...
301     42E               1
302     42F               1
303     42G               4
304     42H               2
305     42I               1

[306 rows x 2 columns]


In [59]:
# Proportion of squirrels eating aboveground vs on the ground 
eating_counts = (results_df[results_df['eating'] == True].groupby('above_ground_sighter')['eating'].count().reset_index(name='eating_squirrel_count'))
print(eating_counts)


   above_ground_sighter  eating_squirrel_count
0                     1                      1
1                    10                      6
2                    12                      1
3                    13                      1
4                    15                      7
5                    18                      1
6                     2                      4
7                    25                      4
8                     3                      4
9                    30                      2
10                   35                      1
11                    4                      3
12                   40                      2
13                   50                      1
14                    6                      2
15                    7                      1
16                    8                      1
17                FALSE                    203


In [ ]:
# Number of activities squirrel is doing at once
results_df['activity_score'] = results_df[['running', 'chasing', 'climbing', 'eating', 'foraging']].sum(axis=1)
print(results_df[['unique_squirrel_id','activity_score', 'running', 'chasing', 'climbing', 'eating', 'foraging']])

    unique_squirrel_id  activity_score  running  chasing  climbing  eating  \
0       37F-PM-1014-03               0    False    False     False   False   
1       21B-AM-1019-04               0    False    False     False   False   
2       11B-PM-1014-08               1    False     True     False   False   
3       32E-PM-1017-14               2    False    False     False    True   
4       13E-AM-1017-05               1    False    False     False   False   
..                 ...             ...      ...      ...       ...     ...   
995     16D-AM-1007-01               2    False    False      True    True   
996      3B-AM-1010-10               2    False    False     False    True   
997     33D-AM-1017-03               1    False    False     False   False   
998      2A-PM-1019-04               1     True    False     False   False   
999      2A-AM-1010-08               2     True    False     False   False   

     foraging  
0       False  
1       False  
2       False  